In [1]:
# Run the script directly from the notebook using IPython magic
%run reconstruct_from_latents.py cfg.py --device cuda


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


c:\Users\ryan\miniconda3\envs\autoencoder\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ryan\miniconda3\envs\autoencoder\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: c:\Users\ryan\miniconda3\envs\autoencoder\lib\site-packages\lpips\weights\v0.1\alex.pth


TypeError: Cannot handle this data type: (1, 1, 16), |u1

In [5]:
# Install Python libraries
%pip install constriction opencv-python ffmpeg-python


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [7]:
import tarfile, os

tar_path    = r"C:\Users\ryan\ece539_Project\ffmpeg-7.1.1.tar.xz"
extract_dir = r"C:\Users\ryan\ece539_Project\ffmpeg"

os.makedirs(extract_dir, exist_ok=True)
with tarfile.open(tar_path, "r:xz") as tar:
    tar.extractall(path=extract_dir)



In [8]:
import os
os.environ["PATH"] += os.pathsep + r"C:\Users\ryan\ece539_Project\ffmpeg\ffmpeg-7.1.1\bin"


In [10]:
import os

base = r"C:\Users\ryan\ece539_Project\ffmpeg"
print("Top-level:", os.listdir(base))
for entry in os.listdir(base):
    full = os.path.join(base, entry)
    if os.path.isdir(full):
        print(f"{entry}/ →", os.listdir(full))


Top-level: ['ffmpeg-7.1.1']
ffmpeg-7.1.1/ → ['.mailmap', 'Changelog', 'compat', 'configure', 'CONTRIBUTING.md', 'COPYING.GPLv2', 'COPYING.GPLv3', 'COPYING.LGPLv2.1', 'COPYING.LGPLv3', 'CREDITS', 'doc', 'ffbuild', 'fftools', 'INSTALL.md', 'libavcodec', 'libavdevice', 'libavfilter', 'libavformat', 'libavutil', 'libpostproc', 'libswresample', 'libswscale', 'LICENSE.md', 'MAINTAINERS', 'Makefile', 'presets', 'README.md', 'RELEASE', 'RELEASE_NOTES', 'tests', 'tools', 'VERSION']


In [11]:
# Note: use %conda, not %pip, to get the actual ffmpeg executable
%conda install -y -c conda-forge ffmpeg


Channels:
 - conda-forge
 - defaults
 - nvidia
 - pytorch
Platform: win-64
Solving environment: ...working... done

## Package Plan ##

  environment location: c:\Users\ryan\miniconda3\envs\autoencoder

  added / updated specs:
    - ffmpeg


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libavif_api-1.1.1         |       h57928b3_3          11 KB  conda-forge
    _openmp_mutex-4.5          |            2_gnu          48 KB  conda-forge
    blosc-1.21.5               |       hdccc3a2_0          49 KB  conda-forge
    c-blosc2-2.12.0            |       h183a6f4_0         204 KB  conda-forge
    ca-certificates-2025.4.26  |       h4c7d964_0         149 KB  conda-forge
    cairo-1.18.4               |       h5782bbf_0         1.5 MB  conda-forge
    certifi-2025.1.31          |     pyhd8ed1ab_0         159 KB  conda-forge
    cfitsio-4.0.0              |       hd67004f_0         560 KB  c

In [12]:
!where ffmpeg
!ffmpeg -version


C:\Users\ryan\miniconda3\envs\autoencoder\Library\bin\ffmpeg.exe
ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
built with clang version 19.1.0
configuration: --prefix=/d/bld/ffmpeg_1728332459246/_h_env/Library --cc=clang.exe --cxx=clang++.exe --nm=llvm-nm --ar=llvm-ar --disable-doc --enable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --ld=lld-link --target-os=win64 --enable-cross-compile --toolchain=msvc --host-cc=clang.exe --extra-libs=ucrt.lib --extra-libs=vcruntime.lib --extra-libs=oldnames.lib --strip=llvm-strip --disable-stripping --host-extralibs= --disable-libopenvino --enable-gpl --enable-libx264 --enable-libx265 --enable-libaom --enable-libsvtav1 --enable-libxml2 --enable-pic --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libopus --pkg-config=/d/bld/ffmpeg_1728332459246/_build_env/Library/bin/pkg-config
libavutil     

In [13]:
import os
import torch
import numpy as np
import cv2
import importlib.machinery
import importlib.util
import constriction


def load_cfg(path):
    loader = importlib.machinery.SourceFileLoader("cfg", path)
    spec = importlib.util.spec_from_loader(loader.name, loader)
    mod = importlib.util.module_from_spec(spec)
    loader.exec_module(mod)
    return mod


def reconstruct_video_from_latents(cfg_path="cfg.py", device="cuda", out_path="reconstructed.avi", fps=24):
    # Set up
    device = torch.device("cuda" if torch.cuda.is_available() and "cuda" in device else "cpu")
    cfg = load_cfg(cfg_path)
    model = cfg.get_model().to(device).eval()
    loader = cfg.get_loader()
    os.makedirs("videos", exist_ok=True)

    LOW, HIGH = -127, 128
    raw_frames = []
    recon_frames = []

    for batch_idx, batch in enumerate(loader):
        batch = batch.to(device)  # shape: (B, C, T, H, W)
        B, C, T, H, W = batch.shape

        # Encode
        if hasattr(model, "encode"):
            mu, logvar = model.encode(batch)
        else:
            out = model(batch)
            mu, logvar = out[1], out[2] if isinstance(out, tuple) else (out["mu"], out["logvar"])

        # Quantize
        means = mu.detach().cpu().numpy().astype(np.float32)
        stds = (0.5 * logvar).exp().detach().cpu().numpy().astype(np.float32)
        ints = np.round(means).astype(np.int32).flatten()

        # Compress
        qmodel = constriction.stream.model.QuantizedGaussian(LOW, HIGH)
        encoder = constriction.stream.queue.RangeEncoder()
        encoder.encode(ints, qmodel, means.flatten(), stds.flatten())
        compressed = encoder.get_compressed()

        # Decompress
        decoder = constriction.stream.queue.RangeDecoder(compressed)
        decoded_ints = decoder.decode(qmodel, means.flatten(), stds.flatten())
        decoded = np.array(decoded_ints, dtype=np.int32).reshape(means.shape)
        z = torch.from_numpy(decoded).view_as(mu).to(device).type_as(mu)

        # Decode
        if hasattr(model, "decode"):
            recon = model.decode(z)
        else:
            out = model(z)
            recon = out[0] if isinstance(out, tuple) else out["recon"]

        # batch frames
        for i in range(batch.shape[0]):
            orig_video = batch[i].permute(1, 2, 3, 0).cpu().numpy()   # (T, H, W, C)
            recon_video = recon[i].permute(1, 2, 3, 0).detach().cpu().numpy()  # (T, H, W, C)

            raw_frames.extend(orig_video)
            recon_frames.extend(recon_video)


    # Save side-by-side video
    height, width = raw_frames[0].shape[:2]
    video_out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (width * 2, height))
    for orig, rec in zip(raw_frames, recon_frames):
        orig = (orig * 255 if orig.max() <= 1 else orig).astype(np.uint8)
        rec = (np.clip(rec, 0, 1) * 255).astype(np.uint8)
        combined = np.hstack((orig, rec))
        video_out.write(cv2.cvtColor(combined, cv2.COLOR_RGB2BGR))

    video_out.release()
    print(f"[✓] Saved video comparison: {out_path}")

# Run it
reconstruct_video_from_latents(cfg_path="cfg.py", out_path="videos/reconstructed.avi")


[✓] Saved video comparison: videos/reconstructed.avi


In [14]:
import subprocess
from IPython.display import HTML

# 1) Paths
avi_path = "videos/reconstructed.avi"        # change if yours is different
mp4_path = "videos/reconstructed.mp4"

# 2) Convert AVI → MP4
#    libx264 gives good compression; crf=23 is a reasonable quality/size trade-off
subprocess.run([
    "ffmpeg", "-y",
    "-i", avi_path,
    "-vf", "scale=iw*2:ih*2",     # optional: double the resolution
    "-vcodec", "libx264",
    "-crf", "23",
    mp4_path
], check=True)

# 3) Display inline
HTML(f"""
<video width=640 controls>
  <source src="{mp4_path}" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")


In [17]:
import os, glob

print("Contents of ./single_video_test/:")
print(os.listdir("./single_video_test"))
print("  mp4 files:", glob.glob("./single_video_test/*.mp4"))


Contents of ./single_video_test/:
['bruh.avi.mp4']
  mp4 files: ['./single_video_test\\bruh.avi.mp4']


In [20]:
import os, torch, numpy as np, cv2, importlib.machinery, importlib.util, constriction
from IPython.display import HTML

# ─── load your cfg.py so we can call get_model() ───────────────────────────────
def load_cfg(path="cfg.py"):
    loader = importlib.machinery.SourceFileLoader("cfg", path)
    spec   = importlib.util.spec_from_loader(loader.name, loader)
    mod    = importlib.util.module_from_spec(spec)
    loader.exec_module(mod)
    return mod

cfg   = load_cfg("cfg.py")
model = cfg.get_model().to("cuda").eval()

# ─── helper to read every frame and normalize ──────────────────────────────────
def extract_video_frames(video_path, size):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, f = cap.read()
        if not ret: break
        f = cv2.resize(f, (size, size))
        f = cv2.cvtColor(f, cv2.COLOR_BGR2RGB) / 255.0
        frames.append(f)
    cap.release()
    return np.stack(frames)  # (F, H, W, 3)

# ─── sliding‐window into length‐L clips, stride=1 ─────────────────────────────
def make_sequences(frames, L):
    return np.stack([frames[i:i+L] for i in range(len(frames)-L+1)])  # (F-L+1, L, H, W, 3)

# ─── full reconstruct + side-by-side video writer ─────────────────────────────
def reconstruct_entire_video(
    video_path,
    cfg,
    model,
    clip_len=16,
    size=256,
    batch_size=1,       # ← default now 1
    device="cuda",
    out_avi="comparison_full.avi",
    fps=24
):
    # 1) grab all raw frames
    raw = extract_video_frames(video_path, size)
    seqs = make_sequences(raw, clip_len)
    N, L, H, W, C = seqs.shape

    # 2) compress / decompress each sliding‐window one at a time
    LOW, HIGH = -127, 128
    qmodel = constriction.stream.model.QuantizedGaussian(LOW, HIGH)
    recon_seqs = []

    for i in range(N):
        batch = seqs[i:i+1]  # (1, L, H, W, C)
        x = torch.from_numpy(batch).permute(0,4,1,2,3).to(device).float()

        # encode
        mu, logvar = model.encode(x)
        means = mu.detach().cpu().numpy().astype(np.float32)
        stds  = (0.5 * logvar).exp().detach().cpu().numpy().astype(np.float32)
        ints  = np.round(means).astype(np.int32).flatten()

        # compress
        enc = constriction.stream.queue.RangeEncoder()
        enc.encode(ints, qmodel, means.flatten(), stds.flatten())
        comp = enc.get_compressed()

        # decompress
        dec = constriction.stream.queue.RangeDecoder(comp)
        decoded = np.array(dec.decode(qmodel, means.flatten(), stds.flatten()),
                           dtype=np.int32).reshape(means.shape)
        z = torch.from_numpy(decoded).view_as(mu).to(device).type_as(mu)

        # decode
        out = model.decode(z)
        if isinstance(out, tuple): out = out[0]
        recon_np = out.permute(0,2,3,4,1).cpu().detach().numpy()  # (1, L, H, W, C)
        recon_seqs.append(recon_np)

    recon_seqs = np.concatenate(recon_seqs, axis=0)  # (N, L, H, W, C)

    # 3) re-stitch: first frame + last frame of every window
    rec = [ recon_seqs[0,0] ] + [ recon_seqs[i,-1] for i in range(N) ]
    rec = np.stack(rec)  # (F, H, W, C)
    raw = raw[:len(rec)]

    # 4) write out side-by-side AVI
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    vw = cv2.VideoWriter(out_avi, fourcc, fps, (W*2, H))
    for o, r in zip(raw, rec):
        o8 = (o*255).astype(np.uint8)
        r8 = (np.clip(r,0,1)*255).astype(np.uint8)
        frame = cv2.cvtColor(np.hstack((o8,r8)), cv2.COLOR_RGB2BGR)
        vw.write(frame)
    vw.release()
    print(f"[✓] Wrote full comparison AVI: {out_avi}")

    # 5) convert → mp4 and embed
    mp4 = out_avi.replace(".avi",".mp4")
    os.system(f'ffmpeg -y -i "{out_avi}" -vcodec libx264 -crf 23 "{mp4}"')
    return HTML(f'<video width=640 controls><source src="{mp4}" type="video/mp4"></video>')

# ─── run it ────────────────────────────────────────────────────────────────────
video_file = "./single_video_test/bruh.avi.mp4"
display = reconstruct_entire_video(
    video_file,
    cfg, model,
    clip_len=16,
    size=256,
    batch_size=1,      # ← invoke with batch_size=1
    device="cuda",
    fps=24
)
display


OutOfMemoryError: CUDA out of memory. Tried to allocate 272.00 MiB. GPU 